In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imread
%matplotlib inline
import os
from dataset import Dataset, default_inception_transform
import numpy as np
import torch
import torchvision
import torch.utils.data as data
from torchvision import datasets, transforms

In [2]:
!python run_attack_iter.py \
  --input_dir=imgs \
  --output_dir=advimgs \
  --max_epsilon=10 \
  --steps 50 \
  --norm 1 \
  --checkpoint_path=inception_v3_google-1a9a5a14.pth

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [3]:
model = torchvision.models.inception_v3(pretrained=False, transform_input=False)
model=model.cuda()
checkpoint = torch.load('inception_v3_google-1a9a5a14.pth')
model.load_state_dict(checkpoint)

In [4]:
out_origs=[]
dataset = Dataset('imgs', target_file='', transform=default_inception_transform(299))
loader = data.DataLoader(dataset, shuffle=False)
model.eval()
for batch_idx, (input, target) in enumerate(loader):
        input = input.cuda()
        target = target.cuda()
        out_origs.append(torch.argmax(model.forward(input)).detach().cpu().numpy())

/opt/conda/lib/python3.6/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
dataset = Dataset('advimgs', target_file='', transform=transforms.ToTensor())
loader = data.DataLoader(dataset, shuffle=False)
out_advs=[]
for batch_idx, (input, target) in enumerate(loader):
        input = input.cuda()
        target = target.cuda()
        out_advs.append(torch.argmax(model.forward(input)).detach().cpu().numpy())

In [7]:
success_rate = 0
for i in range(len(out_origs)):
    if(out_origs[i]!=out_advs[i]):
        success_rate += 1
print(success_rate*100.0/len(out_origs))

100.0
